In [10]:
from nltk import word_tokenize
import json
import re
import logging

# infiles_data_final = {'train': '/Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split/spider_combined_train.json',   
#                 'dev':'/Users/aishwarya/Downloads/spring23/cs685-NLP/project/spider_data/dev.json',
#                 'schema': '/Users/aishwarya/Downloads/spring23/cs685-NLP/project/spider_data/tables.json',
#                 'test': '/Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split/spider_combined_test.json'
# }


infiles_data_final = {'train': '/Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split/spider_train.json',   
                'dev':'/Users/aishwarya/Downloads/spring23/cs685-NLP/project/spider_data/dev.json',
                'schema': '/Users/aishwarya/Downloads/spring23/cs685-NLP/project/spider_data/tables.json',
                'test': '/Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split/spider_test.json'
}

prefix_data_final = '/Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/baseline'
infiles = {}
infiles["data_final"] = [infiles_data_final, prefix_data_final]  


In [11]:
VALUE_NUM_SYMBOL = "{value}"

def strip_table(table):
    column_types = table['column_types']
    table_names_original = [cn.lower() for cn in table['table_names_original']]
    table_names = [cn.lower() for cn in table['table_names']]
    column_names = [cn.lower() for i, cn in table['column_names']]
    column_names_original = [cn.lower() for i, cn in table['column_names_original']]
    return [table_names_original, table_names, column_names_original, column_names, column_types]

def strip_nl(nl):
    '''
    return keywords of nl query
    '''
    nl_keywords = []
    nl = nl.strip()
    nl = nl.replace(";"," ; ").replace(",", " , ").replace("?", " ? ").replace("\t"," ")
    nl = nl.replace("(", " ( ").replace(")", " ) ")
    
    str_1 = re.findall("\"[^\"]*\"", nl)
    str_2 = re.findall("\'[^\']*\'", nl)
    float_nums = re.findall("[-+]?\d*\.\d+", nl)
    
#     values = str_1 + str_2 + float_nums
#     for val in values:
#         nl = nl.replace(val.strip(), VALUE_NUM_SYMBOL)
    
    
    raw_keywords = nl.strip().split()
    for tok in raw_keywords:
        if "." in tok:
            to = tok.replace(".", " . ").split()
            to = [t.lower() for t in to if len(t)>0]
            nl_keywords.extend(to)
        elif "'" in tok and tok[0]!="'" and tok[-1]!="'":
            to = word_tokenize(tok)
            to = [t.lower() for t in to if len(t)>0]
            nl_keywords.extend(to)      
        elif len(tok) > 0:
            nl_keywords.append(tok.lower())
    return nl_keywords


def strip_query(query):
    '''
    return keywords of sql query
    '''
    query_keywords = []
    query = query.strip().replace(";","").replace("\t","")
    query = query.replace("(", " ( ").replace(")", " ) ")
    query = query.replace(">=", " >= ").replace("<=", " <= ").replace("!=", " != ").replace("=", " = ")

    
    # then replace all stuff enclosed by "" with a numerical value to get it marked as {VALUE}
    str_1 = re.findall("\"[^\"]*\"", query)
    str_2 = re.findall("\'[^\']*\'", query)
    
#     values = str_1 + str_2
#     for val in values:
#         query = query.replace(val.strip(), VALUE_NUM_SYMBOL)

    query_tokenized = query.split()
    float_nums = re.findall("[-+]?\d*\.\d+", query)
#     query_tokenized = [VALUE_NUM_SYMBOL if qt in float_nums else qt for qt in query_tokenized]
    query = " ".join(query_tokenized)
    int_nums = [i.strip() for i in re.findall("[^tT]\d+", query)]

    
#     query_tokenized = [VALUE_NUM_SYMBOL if qt in int_nums else qt for qt in query_tokenized]
    # print int_nums, query, query_tokenized
    
    for tok in query_tokenized:
        if "." in tok:
            table = re.findall("[Tt]\d+\.", tok)
            if len(table)>0:
                to = tok.replace(".", " . ").split()
                to = [t.lower() for t in to if len(t)>0]
                query_keywords.extend(to)
            else:
                query_keywords.append(tok.lower())

        elif len(tok) > 0:
            query_keywords.append(tok.lower())
    query_keywords = [w for w in query_keywords if len(w)>0]
    query_sentence = " ".join(query_keywords)
    query_sentence = query_sentence.replace("> =", ">=").replace("! =", "!=").replace("< =", "<=")
#     if '>' in query_sentence or '=' in query_sentence:
#        print query_sentence
    return query_sentence.split()

def count_databases(infile_group):
    content = set()
    with open(infile_group['dev']) as f:
        ex_list = json.load(f)
        for table_dict in ex_list:
            content.add(table_dict["db_id"])
    dev_count = len(content)
    print("the number of dev tables are", dev_count)
    
    with open(infile_group['train']) as f:
        ex_list = json.load(f)
        for table_dict in ex_list:
            content.add(table_dict["db_id"])
    train_count = len(content) - dev_count
    print("the number of train tables are",train_count)
    
    with open(infile_group['test']) as f: 
        ex_list = json.load(f)
        for table_dict in ex_list:
            db_id = table_dict["db_id"]
            if db_id not in table_dict:
                content.add(db_id)
    print("the number of total tables are", len(content))
    return content

def output_vocab_to_txt(outfile, cnt, min_frequency=0, max_vocab_size=None):
    file_obj = open(outfile, 'w')
    # output vocab with min_frequency, but the same weight
    logging.info("Found %d unique tokens in the vocabulary.", len(cnt))
    
    # Filter tokens below the frequency threshold
    fout_decode_file = open(outfile, 'w')
    if min_frequency > 0:
        filtered_tokens = [(w, c) for w, c in cnt.most_common()
                            if c > min_frequency]
        cnt = collections.Counter(dict(filtered_tokens))

        logging.info("Found %d unique tokens with frequency > %d.",
                     len(cnt), min_frequency)

    # Sort tokens by 1. frequency 2. lexically to break ties
    word_with_counts = cnt.most_common()
    word_with_counts = sorted(
        word_with_counts, key=lambda x: (x[1], x[0]), reverse=True)
    # Take only max-vocab
    if max_vocab_size is not None:
        word_with_counts = word_with_counts[:max_vocab_size]

    all_words = {}

    for word, count in word_with_counts:
        try:
            word = str(word)
            if word.strip() in all_words:
                pass
            else:
                all_words[word] = 1
                file_obj.write("{}\t{}\n".format(word, 1))                
        except:
                file_obj.write("{}\t{}\n".format(word.encode('utf-8'), 1))              
    file_obj.close()
    

In [12]:
import sys
import argparse
import collections
import json
import os
from nltk.tokenize import word_tokenize
import re
# from utils import output_vocab_to_txt, infiles, count_databases, strip_query, strip_nl

# parser = argparse.ArgumentParser(
#     description="Generate vocabulary for a tokenized text file.")
# parser.add_argument(
#     "--min_frequency",
#     dest="min_frequency",
#     type=int,
#     default=0,
#     help="Minimum frequency of a word to be included in the vocabulary.")
# parser.add_argument(
#     "--max_vocab_size",
#     dest="max_vocab_size",
#     type=int,
#     help="Maximum number of tokens in the vocabulary")

# args = parser.parse_args()

In [13]:
def get_encode_Query(infile_group, outfile, infile, output=False):
    max_nl = 0
    cnt = collections.Counter()
    infile = infile_group[infile]
    if output:
        outfile = open(outfile, 'w')
    
    cnt = collections.Counter()    
    with open(infile) as f:
        ex_list = json.load(f)
        for nl_sql_dict in ex_list:
            tokens = strip_nl(nl_sql_dict["question"])                    
            cnt.update(tokens)
            max_nl = max(max_nl, len(tokens))
            token_sentence = " ".join(tokens)
            if output:
                try:
                    outfile.write("{}\n".format(token_sentence))
                except:
                    outfile.write("{}\n".format(token_sentence.encode('utf-8')))
                
        if output:
            outfile.close()
    print("max length nl of", infile, "is", max_nl)
    return cnt


def get_mask(infile_group, outfile, infile, vocabfile, output=True):
    
    
    
    _, used_databases, db_dict_rev = get_schema_vocab(infile_group, "schema")
    key_words = sql_key_words()
    infile_name = infile_group[infile]
    if output:
        outfile = open(outfile, 'w')
    vocab_ls = []
    with open(vocabfile, "r") as vocab:
        for line in vocab:
            items = line.split()
            vocab_ls.append(items[0])
    print(f"decode vocab length is {vocabfile}", len(vocab_ls))
    with open(infile_name) as f:
        ex_list = json.load(f)
        for nl_sql_dict in ex_list:
            if infile != 'train':
                db_id = nl_sql_dict["db_id"]

                binary = []
                for item in vocab_ls:
                    # print db_id
                    if item in key_words:
                        binary.append("1")
                    elif (item in db_dict_rev ) and (db_id in db_dict_rev[item]):
                        binary.append("1")
                    else:
                        binary.append("0")
                binary.extend(["1"] * 5)
                if output:
                    outfile.write("{}\n".format(" ".join(binary)))
            elif output:
                outfile.write("{}\n".format("1"))
                

        if output:
            outfile.close()
        
    
        
def get_decode_SQL(infile_group, outfile, infile, output=False, outputdb=False):
    max_sql = 0
    
    cnt = collections.Counter()
    infile = infile_group[infile]
    if output:
        outfile = open(outfile, 'w')
    
    with open(infile) as f:
        ex_list = json.load(f)
        for nl_sql_dict in ex_list:
            tokens = strip_query(nl_sql_dict["query"])       
            max_sql = max(max_sql, len(tokens))
            cnt.update(tokens)
            token_sentence = " ".join(tokens)
            if output and not outputdb:
                try:
                    outfile.write("{}\n".format(token_sentence))
                except:
                    outfile.write("{}\n".format(token_sentence.encode('utf-8')))
            elif output and outputdb:
                try:
                    outfile.write("{}\t{}\n".format(nl_sql_dict["query"].lower().replace("\t", " "), nl_sql_dict["db_id"]))
                except:
                    outfile.write("{}\t{}\n".format(nl_sql_dict["query"].encode('utf-8').lower().replace("\t", " "), nl_sql_dict["db_id"]))
        if output:
            outfile.close()
    print("max sql length of", infile, "is", max_sql)
    return cnt

def get_schema_vocab(infile_group, infile):
    used_databases = set()
    cnt = collections.Counter()
    db_dict_rev = {}
    with open(infile_group[infile]) as f:
        ex_list = json.load(f)
        for table_dict in ex_list:
            db_id = table_dict["db_id"]
            if db_id not in used_databases:
                used_databases.add(db_id)
            new_tokens = []
            column_names = table_dict["column_names_original"]
            table_names = table_dict["table_names_original"]
            for item in column_names:
                new_tokens.append(item[1].lower())
            for table_name in table_names:
                new_tokens.append(table_name.lower())
            cnt.update(new_tokens)   
            
            # build look up
            for tok in new_tokens:
                if tok not in db_dict_rev:
                    db_dict_rev[tok] = [db_id]
                else:
                    if db_id not in db_dict_rev[tok]:
                        db_dict_rev[tok].append(db_id)
                    
                
    return cnt, used_databases, db_dict_rev



def sql_key_words():
    cnt = collections.Counter()
    cnt.update(["t"+str(i+1) for i in range(10)])
    cnt.update([".", ",", "(", ")", "in", "not", "and", "between", "or", "where",
                "except", "union", "intersect",
                "group", "by", "order", "limit", "having","asc", "desc",
                "count", "sum", "avg", "max", "min",
                "{value}",
               "<", ">", "=", "!=", ">=", "<=",
                "like",
                "distinct","*",
                "join", "on", "as", "select", "from"
               ])
    return cnt


def get_decode_vocab_no_weight(infile_group, outfile):

    cnt, _, db_dict_rev = get_schema_vocab(infile_group, "schema")
    cnt.update(sql_key_words())
    output_vocab_to_txt(outfile, cnt)



def get_encode_vocab_no_weight(infile_group, outfile):
    cnt = collections.Counter()
    cnt.update(get_encode_Query(infile_group, None, 'train', False))
    cnt.update(get_encode_Query(infile_group, None, 'dev', False))   
    cnt.update(get_schema_vocab(infile_group, "schema")[0])
    output_vocab_to_txt(outfile, cnt)

    
def decode_encode_copy(infile_group, outfile):
    cnt = collections.Counter()

    cnt.update(get_encode_Query(infile_group, None, 'train', False))
    cnt.update(get_encode_Query(infile_group, None, 'dev', False))   
    cnt.update(get_schema_vocab(infile_group, "schema")[0])
    cnt.update(sql_key_words())

    output_vocab_to_txt(outfile, cnt)


In [14]:
for key in infiles.keys():
    infile_group, prefix = infiles[key]
    count_databases(infile_group)
    if not os.path.exists(os.path.join(prefix,'dev')):
        os.makedirs(os.path.join(prefix,'dev'))
    if not os.path.exists(os.path.join(prefix,'train')):
        os.makedirs(os.path.join(prefix,'train'))
    if not os.path.exists(os.path.join(prefix,'test')):
        os.makedirs(os.path.join(prefix,'test'))

    get_decode_SQL(infile_group, os.path.join(prefix,'dev', "dev_decode.txt"), "dev", True)        
    get_decode_SQL(infile_group, os.path.join(prefix,'test', "test_decode.txt"), "test", True)
    get_decode_SQL(infile_group, os.path.join(prefix,'train', "train_decode.txt"), "train", True)

    get_decode_SQL(infile_group, os.path.join(prefix,'dev', "dev_decode_db.txt"), "dev", True, True)       
    get_decode_SQL(infile_group, os.path.join(prefix,'test', "test_decode_db.txt"), "test", True, True)
    get_decode_SQL(infile_group, os.path.join(prefix,'train', "train_decode_db.txt"), "train", True, True)

    get_encode_Query(infile_group, os.path.join(prefix,'test', "test_encode.txt"), "test", True)
    get_encode_Query(infile_group, os.path.join(prefix,'dev', "dev_encode.txt"), "dev", True)
    get_encode_Query(infile_group, os.path.join(prefix,'train', "train_encode.txt"), "train", True)

    get_encode_vocab_no_weight(infile_group, os.path.join(prefix, "encode_vocab.txt"))

    get_decode_vocab_no_weight(infile_group, os.path.join(prefix, "decode_vocab.txt"))
    get_decode_vocab_no_weight(infile_group, os.path.join(prefix, "decode_copy_encode_vocab.txt"))


    get_mask(infile_group, os.path.join(prefix, "test", "test_decoder_mask.txt"), "test",os.path.join(prefix, "decode_vocab.txt"), True)
    get_mask(infile_group, os.path.join(prefix, "dev", "dev_decoder_mask.txt"), "dev",os.path.join(prefix, "decode_vocab.txt"), True)
    get_mask(infile_group, os.path.join(prefix, "train", "train_decoder_mask.txt"), "train", os.path.join(prefix, "decode_vocab.txt"),True)

the number of dev tables are 20
the number of train tables are 129
the number of total tables are 160
max sql length of /Users/aishwarya/Downloads/spring23/cs685-NLP/project/spider_data/dev.json is 88
max sql length of /Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split/spider_test.json is 81
max sql length of /Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split/spider_train.json is 127
max sql length of /Users/aishwarya/Downloads/spring23/cs685-NLP/project/spider_data/dev.json is 88
max sql length of /Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split/spider_test.json is 81
max sql length of /Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split/spider_train.json is 127
max length nl of /Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split/spider_test.json is 39
max length nl of /Users/aishwarya/Downloads/spring23/cs685-NLP/project/spider_data/dev.json is 33
max length nl of /Users/aishwarya/Downloads/spring23/cs685-NLP/projec